In [2]:
import pandas as pd
import numpy as np

In [3]:
data_frame = pd.read_csv("./train.csv")

In [7]:
from sklearn.model_selection import train_test_split

In [11]:
X = data_frame.drop('Survived', axis=1)

In [12]:
Y = data_frame['Survived']

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, train_size=0.75)

In [18]:
X_train.shape

(668, 11)

In [19]:
Y_train.shape

(668,)

In [20]:
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
424,425,3,"Rosblom, Mr. Viktor Richard",male,18.0,1,1,370129,20.2125,NaN,S
50,51,3,"Panula, Master. Juha Niilo",male,7.0,4,1,3101295,39.6875,NaN,S
799,800,3,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",female,30.0,1,1,345773,24.1500,NaN,S
234,235,2,"Leyson, Mr. Robert William Norman",male,24.0,0,0,C.A. 29566,10.5000,NaN,S
828,829,3,"McCormack, Mr. Thomas Joseph",male,NaN,0,0,367228,7.7500,NaN,Q


In [22]:
X_train.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,668.000000,668.000000,543.000000,668.000000,668.000000,668.000000
mean,443.853293,2.285928,29.817993,0.523952,0.368263,32.239246
std,255.571728,0.839779,14.435116,1.139858,0.787274,48.821483
min,2.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,225.750000,1.750000,20.500000,0.000000,0.000000,7.925000
50%,441.500000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,661.250000,3.000000,38.000000,1.000000,0.000000,31.275000
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


**Because we don't have many columns, we can see each column's dependency on the 'Survived' target.**

In [39]:
X_train.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            125
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          513
Embarked         2
dtype: int64

In [24]:
import matplotlib.pyplot as plt

In [37]:
X_train[X_train['Cabin'].isna()==False]['Cabin']

102            D26
835            E49
291            B49
872    B51 B53 B55
438    C23 C25 C27
          ...     
54             B30
96              A5
23              A6
139            B86
782             D6
Name: Cabin, Length: 155, dtype: object